In [984]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as plt
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [985]:
#cargar archivo desde seaborn 
df=pd.read_csv('Datos_clean_BO.csv', encoding= 'latin1')
#eliminamos la columna unnamed 
df= df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df.head(5)

,source,name,description,host_name,host_since,host_location,host_response_time,host_is_superhost,host_identity_verified,neighbourhood_cleansed,...,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,city scrape,50 sm Studio in the historic centre,The studio is a fully restored ancient basemen...,Carlo,2010-07-29,"Milan, Italy",within an hour,f,t,Santo Stefano,...,15.0,1.0,4.64,4.77,4.65,4.92,4.88,4.87,4.66,1.37
1,city scrape,A room in Pasolini's house,"Simple, cozy and silent room in a lived house ...",Eleonora,2011-03-28,"Bologna, Italy",a few days or more,f,t,Porto - Saragozza,...,47.0,1.0,4.72,4.74,4.60,4.83,4.85,4.84,4.67,2.57
2,city scrape,Garden House Bologna,Sin descripciÃÂ³n,Anna Maria,2011-03-28,"Bologna, Italy",Sin tiempo,f,t,Santo Stefano,...,7.0,0.0,4.91,4.92,4.83,4.98,4.98,4.80,4.81,0.39
3,city scrape,Private Room & Bath Ex-Convent in Centro Storico,The neighbourhood is the most prestigious of B...,Elizabeth,2011-05-09,"Bologna, Italy",within an hour,f,t,Porto - Saragozza,...,5.0,0.0,4.90,4.82,4.92,4.94,4.93,4.97,4.86,1.68
4,city scrape,Gorgeous Haven in Centro Storico,"Quiet, bright, fully equipped 800 square feet ...",Elisa,2011-09-10,"Bologna, Italy",within a few hours,t,t,Santo Stefano,...,5.0,0.0,4.84,4.92,4.95,4.90,4.92,4.84,4.78,1.09


In [986]:
#convertimos string a tipos numéricos
#df['host_is_superhost'] = df['host_is_superhost'].replace({'f': 0, 't': 1})
#df['host_identity_verified'] = df['host_identity_verified'].replace({'f': 0, 't': 1})
#df['instant_bookable'] = df['instant_bookable'].replace({'f': 0, 't': 1})

#frecuencias = df['property_type'].value_counts()
#df['property_type'] = df['property_type'].map(frecuencias)


#df['host_response_rate'] = df['host_response_rate'].astype(str).str.rstrip('%')
#df['host_response_rate'] = pd.to_numeric(df['host_response_rate'], errors='coerce').fillna(0)

#df['host_response_time'] = df['host_response_rate'].astype(str).str.rstrip('%')
#df['host_response_time'] = pd.to_numeric(df['host_response_rate'], errors='coerce').fillna(0)

#df['room_type'] = df['room_type'].map({
    #'Entire home/apt': 1,
   # 'Private room': 2,
  # 'Shared room': 3,
 #  'Hotel room': 4
#})

df['price'] = df['price'].astype(str).str.rstrip('$')
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)

In [987]:
df['host_is_superhost'] = df['host_is_superhost'].replace({'True': 1,'False': 0,'t': 1,'f': 0})

C:\Users\pplcf\AppData\Local\Temp\ipykernel_18704\4049798518.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_is_superhost'] = df['host_is_superhost'].replace({'True': 1,'False': 0,'t': 1,'f': 0})


In [988]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   source                       4843 non-null   object 
 1   name                         4843 non-null   object 
 2   description                  4843 non-null   object 
 3   host_name                    4843 non-null   object 
 4   host_since                   4843 non-null   object 
 5   host_location                4843 non-null   object 
 6   host_response_time           4843 non-null   object 
 7   host_is_superhost            4843 non-null   int64  
 8   host_identity_verified       4843 non-null   object 
 9   neighbourhood_cleansed       4843 non-null   object 
 10  property_type                4843 non-null   object 
 11  room_type                    4843 non-null   object 
 12  amenities                    4843 non-null   object 
 13  has_availability  

In [989]:
valor_euro = 18.50
df["price_mxn"] = df["price"] * valor_euro

median_price = df["price_mxn"].median()
df["precio"] = (df["price_mxn"]>median_price).astype(int)

In [990]:
df["room_type"] = df["room_type"].replace({
    'Entire home/apt':1,
    'Private room' : 0,
    'Shared room':0,
    'Hotel room': 0
})

C:\Users\pplcf\AppData\Local\Temp\ipykernel_18704\3362238812.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["room_type"] = df["room_type"].replace({


In [991]:
df['host_response_time'] = df['host_response_time'].replace({'within an hour': 1,
    'within a few hours': 1,
    'within a day': 1,
    'Sin tiempo': 0,
    'a few days or more': 0})

C:\Users\pplcf\AppData\Local\Temp\ipykernel_18704\2965821209.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_response_time'] = df['host_response_time'].replace({'within an hour': 1,


In [992]:
#Convertimos los booleanos a números
df['instant_bookable'] = df['instant_bookable'].replace({'f': 0, 't': 1})

#Convertimos los booleanos a números
df['host_is_superhost'] = df['host_is_superhost'].replace({'f': 0, 't': 1})

C:\Users\pplcf\AppData\Local\Temp\ipykernel_18704\4126096445.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['instant_bookable'] = df['instant_bookable'].replace({'f': 0, 't': 1})


In [993]:
df['host_identity_verified'] = df['host_identity_verified'].replace({ 'f':0, 't': 1, 'False': 0})

C:\Users\pplcf\AppData\Local\Temp\ipykernel_18704\954321434.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_identity_verified'] = df['host_identity_verified'].replace({ 'f':0, 't': 1, 'False': 0})


In [994]:
df['has_availability'] = df['has_availability'].replace({ 'f':0, 't': 1, 'False': 0})

C:\Users\pplcf\AppData\Local\Temp\ipykernel_18704\3395694939.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['has_availability'] = df['has_availability'].replace({ 'f':0, 't': 1, 'False': 0})


In [995]:
median_value = df["accommodates"].median()
df["accommodates_binary"] = (df["accommodates"])>median_value.astype(int)

In [996]:
median_rating = df["review_scores_rating"].median()
df["review_rating"] = (df["review_scores_rating"]>median_rating).astype(int)

In [997]:
df['host_response_rate'] = df['host_response_rate'].astype(str).str.rstrip('%')
df['host_response_rate'] = pd.to_numeric(df['host_response_rate'], errors='coerce').fillna(0)

In [998]:
df['review_scores_rating'] = np.where(
    df['review_scores_rating'] > 4.8,  # Condición
    1,                                  # Valor si verdadero (alta)
    0                                   # Valor si falso (baja)
)

In [999]:
df['accommodates'] = np.where(
    df['accommodates'] > 2,  # Condición (ajusta el número según la media real)
    1,                       # 1 = Mayor a la media (alta capacidad)
    0                        # 0 = Menor o igual a la media (baja capacidad)
)

In [1000]:
df['beds'] = np.where(
    df['beds'] > 2,  # Condición (ajusta el número según la media real)
    1,                       # 1 = Mayor a la media (alta capacidad)
    0                        # 0 = Menor o igual a la media (baja capacidad)
)

In [1001]:
df['price_mxn'] = np.where(
    df['price_mxn'] > 1093,  # Condición (ajusta el valor según tu media real)
    1,                       # 1 = "caro"
    0                        # 0 = "barato"
)

In [1002]:
#df.to_csv('resultados_categorizados.csv', index=False)

Declaración de variables

In [1003]:
#Declaramos las variables dependientes e independientes para la regresión logistica
#1.Instant Bookable
#Vars_Indep1 = df[['host_is_superhost', 'host_response_time', 'price_mxn']]
#Var_Dep1 = df['instant_bookable']

#2.Has availability
#Vars_Indep2 = df[['availability_365', 'minimum_nights', 'price_mxn']]
#Var_Dep2 = df['has_availability']

#3.Host is superhost
#Vars_Indep3 = df[['number_of_reviews', 'review_scores_rating', 'host_response_rate']]
#Var_Dep3 = df['host_is_superhost']

#4.Host identity verified
#Vars_Indep4 = df[['host_response_time', 'host_acceptance_rate', 'host_listings_count']]
#Var_Dep4 = df['host_identity_verified']

#5.Host response time
#Vars_Indep5 = df[['host_response_rate', 'host_is_superhost', 'number_of_reviews']]
#Var_Dep5 = df['host_response_time']

#6.Review scores rating
#Vars_Indep6 = df[['host_is_superhost','number_of_reviews','reviews_per_month']]
#Var_Dep6 = df['review_scores_rating']

#7.accomodates
#Vars_Indep7= df[['room_type', 'price_mxn', 'beds']]
#Var_Dep7 = df['accommodates']

#8.Price
#Vars_Indep8 = df[['room_type', 'accommodates', 'review_scores_rating']]
#Var_Dep8 = df['price_mxn']

#9.Room type
#Vars_Indep9 = df[['accommodates', 'beds', 'price_mxn']]
#Var_Dep9= df['room_type']
 
#10.Availability 365
#Vars_Indep10 = df[['price_mxn', 'accommodates', 'room_type']]
#Var_Dep10 = df['availability_365']

MODELO 1. INSTANT BOOKABLE

In [1004]:
#1.Instant Bookable
Vars_Indep1 = df[['host_is_superhost', 'host_response_time', 'price_mxn']]
Var_Dep1 = df['instant_bookable']

In [1005]:
X = Vars_Indep1
y = Var_Dep1

In [1006]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(Vars_Indep1, Var_Dep1, test_size=0.3, random_state=None)

In [1007]:
#Se escalan todos los datos 
escalar1 = StandardScaler()

In [1008]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba 
X_train1 = escalar1.fit_transform(X_train1)
X_test1 = escalar1.transform(X_test1)

In [1009]:
#Definimos el algoritmo a utilizar 
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()  #le asignamos la importación a una variable

In [1010]:
#Entrenamos el modelo 
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [1011]:
#Realizamos una predicción 
y_pred1 = algoritmo1.predict(X_test1)    
y_pred1 

array([1, 1, 1, ..., 0, 0, 1], shape=(1453,))

In [1012]:
#Verificamos la matriz de Confusión 
from sklearn.metrics import confusion_matrix
matriz1 = confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[191 545]
 [ 92 625]]


In [1013]:
#Calculando la precisión del modelo 
from sklearn.metrics import precision_score

precision1 = precision_score(y_test1, y_pred1, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision1)

Precisión del modelo:
0.6749116607773852


In [1014]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud1 = accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.5615966964900206


In [1015]:
#Calculando la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad1 = recall_score(y_test1, y_pred1, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.2595108695652174


MODELO 2. Has availability

In [1016]:
#valor_euro = 18.50
#df["price_mxn"] = df["price"] * valor_euro

In [1017]:
#2.Has availability
Vars_Indep2 = df[['availability_365', 'minimum_nights', 'price_mxn']]
Var_Dep2 = df['has_availability']

In [1018]:
#verificar los valores sin repetirse de una columna
unico2 = np.unique(df['has_availability'])
unico2

array([0, 1])

In [1019]:
#verificar los valores sin repetirse de una columna
unico2_2 = np.unique(df['minimum_nights'])
unico2_2

array([1. , 1.6, 2. , 3. ])

In [1020]:
#verificar los valores sin repetirse de una columna
unico2_3 = np.unique(df['price_mxn'])
unico2_3

array([0, 1])

In [1021]:
X2 = Vars_Indep2
y2 = Var_Dep2

In [1022]:
X2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   availability_365  4843 non-null   int64  
 1   minimum_nights    4843 non-null   float64
 2   price_mxn         4843 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 113.6 KB


In [1023]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(Vars_Indep2, Var_Dep2, test_size=0.3, random_state=None)

In [1024]:
#Se escalan todos los datos 
escalar2 = StandardScaler()

In [1025]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba 
X_train2 = escalar2.fit_transform(X_train2)
X_test2 = escalar2.transform(X_test2)

In [1026]:
#Definimos el algoritmo a utilizar 
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()  #le asignamos la importación a una variable

In [1027]:
#Entrenamos el modelo 
algoritmo2.fit(X_train2, y_train2)

LogisticRegression()

In [1028]:
#Realizamos una predicción 
y_pred2 = algoritmo2.predict(X_test2)    
y_pred2 

array([1, 1, 1, ..., 1, 1, 1], shape=(1453,))

In [1029]:
#Verificamos la matriz de Confusión 
from sklearn.metrics import confusion_matrix
matriz2 = confusion_matrix(y_test2, y_pred2)
print('Matriz de Confusión:')
print(matriz2)

Matriz de Confusión:
[[   0    8]
 [   0 1445]]


In [1030]:
#Calculando la precisión del modelo 
from sklearn.metrics import precision_score

precision2 = precision_score(y_test2, y_pred2, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision2)

Precisión del modelo:
0.9944941500344116


In [1031]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud2 = accuracy_score(y_test2, y_pred2)
print('Exactitud del modelo:')
print(exactitud2)

Exactitud del modelo:
0.9944941500344116


In [1032]:
#Calculando la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad2 = recall_score(y_test2, y_pred2, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad2)

Sensibilidad del modelo:
0.0


MODELO 3. Host is superhost

In [1033]:
#3.Host is superhost
Vars_Indep3 = df[['number_of_reviews', 'review_scores_rating', 'host_response_rate']]
Var_Dep3 = df['host_is_superhost']

In [1034]:
X3 = Vars_Indep3
y3 = Var_Dep3

In [1035]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(Vars_Indep3, Var_Dep3, test_size=0.3, random_state=None)

In [1036]:
#Se escalan todos los datos 
escalar3 = StandardScaler()

In [1037]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba 
X_train3 = escalar3.fit_transform(X_train3)
X_test3 = escalar3.transform(X_test3)

In [1038]:
#Definimos el algoritmo a utilizar 
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()  #le asignamos la importación a una variable

In [1039]:
#Entrenamos el modelo 
algoritmo3.fit(X_train3, y_train3)

LogisticRegression()

In [1040]:
#Realizamos una predicción 
y_pred3 = algoritmo3.predict(X_test3)    
y_pred3 

array([0, 0, 0, ..., 0, 0, 0], shape=(1453,))

In [1041]:
#Verificamos la matriz de Confusión 
from sklearn.metrics import confusion_matrix
matriz3 = confusion_matrix(y_test3, y_pred3)
print('Matriz de Confusión:')
print(matriz3)

Matriz de Confusión:
[[772 101]
 [253 327]]


In [1042]:
#Calculando la precisión del modelo 
from sklearn.metrics import precision_score

precision3 = precision_score(y_test3, y_pred3, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision3)

Precisión del modelo:
0.7531707317073171


In [1043]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud3 = accuracy_score(y_test3, y_pred3)
print('Exactitud del modelo:')
print(exactitud3)

Exactitud del modelo:
0.7563661390227117


In [1044]:
#Calculando la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad3 = recall_score(y_test3, y_pred3, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad3)

Sensibilidad del modelo:
0.8843069873997709


MODELO 4. HOST IDENTITY VERIFIED

In [1045]:
#4.Host identity verified
Vars_Indep4 = df[['host_response_time', 'host_acceptance_rate', 'host_listings_count']]
Var_Dep4 = df['host_identity_verified']

In [1046]:
X4 = Vars_Indep4
y4 = Var_Dep4

In [1047]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(Vars_Indep4, Var_Dep4, test_size=0.3, random_state=None)

In [1048]:
escalar4 = StandardScaler() #escalan todos los datos

In [1049]:
X_train4 = escalar4.fit_transform(X_train4)
X_test4 = escalar4.transform(X_test4)

In [1050]:
#define el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()

In [1051]:
algoritmo4.fit(X_train4, y_train4)
#entrena el modelo

LogisticRegression()

In [1052]:
y_pred4 = algoritmo4.predict(X_test4)
y_pred4

array([1, 1, 1, ..., 1, 1, 1], shape=(1453,))

In [1053]:
from sklearn.metrics import confusion_matrix
matriz4 = confusion_matrix(y_test4, y_pred4)
print('Matriz de confusión: ')
print(matriz4)

Matriz de confusión: 
[[   0   67]
 [   0 1386]]


In [1054]:
#precision del modelo 
from sklearn.metrics import precision_score

precision4 = precision_score(y_test4, y_pred4, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision4)

Precisión del modelo:
0.9538885065381968


In [1055]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud4 = accuracy_score(y_test4, y_pred4)
print('Exactitud del modelo:')
print(exactitud4)

Exactitud del modelo:
0.9538885065381968


In [1056]:
#Calculando la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad4 = recall_score(y_test4, y_pred4, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad4)

Sensibilidad del modelo:
1.0


MODELO 5. Host response time

In [1057]:
Vars_Indep5 = df[['host_response_rate', 'host_is_superhost', 'number_of_reviews']]
Var_Dep5 = df['host_response_time']

In [1058]:
X5 = Vars_Indep5
y5 = Var_Dep5

In [1059]:
X5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   host_response_rate  4843 non-null   float64
 1   host_is_superhost   4843 non-null   int64  
 2   number_of_reviews   4843 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 113.6 KB


In [1060]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(Vars_Indep5, Var_Dep5, test_size=0.3, random_state=None)

In [1061]:
escalar5 = StandardScaler() #escalan todos los datos

In [1062]:
X_train5 = escalar5.fit_transform(X_train5)
X_test5 = escalar5.transform(X_test5)

In [1063]:
#define el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo5 = LogisticRegression()

In [1064]:
algoritmo5.fit(X_train5, y_train5)
#entrena el modelo

LogisticRegression()

In [1065]:
y_pred5 = algoritmo5.predict(X_test5)
y_pred5

array([1, 1, 1, ..., 1, 1, 1], shape=(1453,))

In [1066]:
from sklearn.metrics import confusion_matrix
matriz5 = confusion_matrix(y_test5, y_pred5)
print('Matriz de confusión: ')
print(matriz5)

Matriz de confusión: 
[[   0  200]
 [   0 1253]]


In [1067]:
#precision del modelo 
from sklearn.metrics import precision_score

precision5 = precision_score(y_test5, y_pred5, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision5)

Precisión del modelo:
0.8623537508602891


In [1068]:
#verificar los valores sin repetirse de una columna
unico5 = np.unique(df['host_is_superhost'])
unico5

array([0, 1])

In [1069]:
from sklearn.metrics import accuracy_score

exactitud5 = accuracy_score(y_test5, y_pred5)
print('Exactitud del modelo:')
print(exactitud5)

Exactitud del modelo:
0.8623537508602891


In [1070]:
from sklearn.metrics import recall_score

sensibilidad5 = recall_score(y_test5, y_pred5,average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad5)

Sensibilidad del modelo:
0.0


MODELO 6. REVIEW SCORES RATING

In [1071]:
#6.Review scores rating
Vars_Indep6 = df[['host_is_superhost','number_of_reviews','reviews_per_month']]
Var_Dep6 = df['review_scores_rating']

In [1072]:
X6 = Vars_Indep6
y6 = Var_Dep6

In [1073]:
X_train6, X_test6, y_train6, y_test6 = train_test_split(Vars_Indep6, Var_Dep6, test_size=0.3, random_state=None)

In [1074]:
escalar6 = StandardScaler() #escalan todos los datos

In [1075]:
X_train6 = escalar6.fit_transform(X_train6)
X_test6 = escalar6.transform(X_test6)

In [1076]:
#define el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo6 = LogisticRegression()

In [1077]:
algoritmo6.fit(X_train6, y_train6)


LogisticRegression()

In [1078]:
y_pred6 = algoritmo6.predict(X_test6)
y_pred6

array([1, 0, 0, ..., 0, 1, 0], shape=(1453,))

In [1079]:
from sklearn.metrics import confusion_matrix
matriz6 = confusion_matrix(y_test6, y_pred6)
print('Matriz de confusión: ')
print(matriz6)

Matriz de confusión: 
[[600 140]
 [304 409]]


In [1080]:
#precision del modelo 
from sklearn.metrics import precision_score

precision6 = precision_score(y_test6, y_pred6, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision6)

Precisión del modelo:
0.6637168141592921


In [1081]:
from sklearn.metrics import accuracy_score

exactitud6 = accuracy_score(y_test6, y_pred6)
print('Exactitud del modelo:')
print(exactitud6)

Exactitud del modelo:
0.6944253269098417


In [1082]:
from sklearn.metrics import recall_score

sensibilidad6 = recall_score(y_test6, y_pred6,average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad6)

Sensibilidad del modelo:
0.8108108108108109


MODELO 7. ACCOMMODATES

In [1083]:
#7.accomodates
Vars_Indep7= df[['room_type', 'price_mxn', 'beds']]
Var_Dep7 = df['accommodates']

In [1084]:
X7 = Vars_Indep7
y7 = Var_Dep7

In [1085]:
X_train7, X_test7, y_train7, y_test7 = train_test_split(Vars_Indep7, Var_Dep7, test_size=0.3, random_state=None)

In [1086]:
escalar7 = StandardScaler() #escalan todos los datos


In [1087]:
X_train7 = escalar7.fit_transform(X_train7)
X_test7 = escalar7.transform(X_test7)

In [1088]:
#define el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo7 = LogisticRegression()

In [1089]:
algoritmo7.fit(X_train7, y_train7)
#entrena el modelo

LogisticRegression()

In [1090]:
y_pred7 = algoritmo7.predict(X_test7)
y_pred7

array([1, 1, 1, ..., 0, 1, 1], shape=(1453,))

In [1091]:
from sklearn.metrics import confusion_matrix
matriz7 = confusion_matrix(y_test7, y_pred7)
print('Matriz de confusión: ')
print(matriz7)

Matriz de confusión: 
[[256 273]
 [ 69 855]]


In [1092]:
#precision del modelo 
from sklearn.metrics import precision_score

precision7 = precision_score(y_test7, y_pred7, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision7)

Precisión del modelo:
0.7876923076923077


In [1093]:
from sklearn.metrics import accuracy_score

exactitud7 = accuracy_score(y_test7, y_pred7)
print('Exactitud del modelo:')
print(exactitud7)

Exactitud del modelo:
0.7646249139710943


In [1094]:
from sklearn.metrics import recall_score

sensibilidad7 = recall_score(y_test7, y_pred7,average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad7)

Sensibilidad del modelo:
0.4839319470699433


MODELO 8. PRICE

In [1095]:
#8.Price
Vars_Indep8 = df[['room_type', 'accommodates', 'review_scores_rating']]
Var_Dep8 = df['price_mxn']

In [1096]:
X8 = Vars_Indep8
y8 = Var_Dep8

In [1097]:
X_train8, X_test8, y_train8, y_test8 = train_test_split(Vars_Indep8, Var_Dep8, test_size=0.3, random_state=None)

In [1098]:
escalar8 = StandardScaler() #escalan todos los datos

In [1099]:
X_train8 = escalar8.fit_transform(X_train8)
X_test8 = escalar8.transform(X_test8)

In [1100]:
#define el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo8 = LogisticRegression()

In [1101]:
algoritmo8.fit(X_train8, y_train8)
#entrena el modelo

LogisticRegression()

In [1102]:
y_pred8 = algoritmo8.predict(X_test8)
y_pred8

array([1, 1, 1, ..., 1, 1, 1], shape=(1453,))

In [1103]:
from sklearn.metrics import confusion_matrix
matriz8 = confusion_matrix(y_test8, y_pred8)
print('Matriz de confusión: ')
print(matriz8)

Matriz de confusión: 
[[   0   99]
 [   0 1354]]


In [1104]:
#precision del modelo 
from sklearn.metrics import precision_score

precision8 = precision_score(y_test8, y_pred8, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision8)

Precisión del modelo:
0.0


C:\Users\pplcf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [1105]:
#verificar los valores sin repetirse de una columna
unico8 = np.unique(df['price_mxn'])
unico8

array([0, 1])

In [1106]:
from sklearn.metrics import accuracy_score

exactitud8 = accuracy_score(y_test8, y_pred8)
print('Exactitud del modelo:')
print(exactitud8)

Exactitud del modelo:
0.9318651066758431


In [1107]:
from sklearn.metrics import recall_score

sensibilidad8 = recall_score(y_test8, y_pred8,average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad8)

Sensibilidad del modelo:
0.0


MODELO 9. ROOM TYPE

In [1108]:
df["room_type"] = df["room_type"].replace({'1':1, '2' : 0,'3':0,'4': 0})

In [1109]:
#9.Room type
Vars_Indep9 = df[['accommodates', 'beds', 'price_mxn']]
Var_Dep9= df['room_type']

In [1110]:
X9 =  Vars_Indep9 #representa arreglo matricial por eso la mayuscula
y9 = Var_Dep9 #arreglo vectorial

In [1111]:
unico9 = np.unique(df['room_type'])
unico9
conteo9 = df["room_type"].value_counts()  # Cuenta por categoría
conteo9

room_type
1    3756
0    1087
Name: count, dtype: int64

In [1112]:
df['price_mxn'] = np.where(
    df['price_mxn'] > 103,  # Condición (ajusta el valor según tu media real)
    1,                       # 1 = "caro"
    0                        # 0 = "barato"
)

In [1113]:
X_train9, X_test9, y_train9, y_test9 = train_test_split(Vars_Indep9, Var_Dep9, test_size=0.3, random_state=None)

In [1114]:
escalar9 = StandardScaler() #escalan todos los datos

In [1115]:
X_train9 = escalar9.fit_transform(X_train9)
X_test9 = escalar9.transform(X_test9)

In [1116]:
#define el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo9 = LogisticRegression()

In [1117]:
algoritmo9.fit(X_train9, y_train9)
#entrena el modelo

LogisticRegression()

In [1118]:
y_pred9 = algoritmo9.predict(X_test9)
y_pred9

array([1, 1, 1, ..., 1, 1, 1], shape=(1453,))

In [1119]:
#Convertimos la variable dependiente a dicotómica
#df['room_type']= df['room_type'].replace(["Hotel room", "Shared room", "Private room"],0)

In [1120]:
from sklearn.metrics import confusion_matrix
matriz9 = confusion_matrix(y_test9, y_pred9)
print('Matriz de confusión: ')
print(matriz9)

Matriz de confusión: 
[[  65  259]
 [   9 1120]]


In [1134]:
#precision del modelo 
from sklearn.metrics import precision_score

precision9 = precision_score(y_test9, y_pred9, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision9)

Precisión del modelo:
0.8783783783783784


In [1122]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud9 = accuracy_score(y_test9, y_pred9)
print('Exactitud del modelo:')
print(exactitud9)

Exactitud del modelo:
0.8155540261527874


In [1123]:
#Calculando la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad9 = recall_score(y_test9, y_pred9, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad9)

Sensibilidad del modelo:
0.9920283436669619


MODELO 10. AVAILABILITY 365

In [1157]:
df['availability_365'] = np.where(
    df['availability_365'] > 180,  # Condición
    1,                              # 1 = Alta disponibilidad (>180 días)
    0                               # 0 = Baja disponibilidad (<=180 días)
)

In [1158]:
unico10 = np.unique(df['availability_365'])
unico10
conteo10 = df["availability_365"].value_counts()  # Cuenta por categoría
conteo10

availability_365
0    2802
1    2041
Name: count, dtype: int64

In [1159]:
#10.Availability 365
Vars_Indep10 = df[['price_mxn', 'accommodates', 'room_type']]
Var_Dep10 = df['availability_365']

In [1166]:
X10 =  Vars_Indep10 #representa arreglo matricial por eso la mayuscula
y10 = Var_Dep10 #arreglo vectorial

In [1167]:
X_train10, X_test10, y_train10, y_test10 = train_test_split(Vars_Indep10, Var_Dep10, test_size=0.3, random_state=None)

In [1168]:
escalar10 = StandardScaler() #escalan todos los datos

In [1169]:
X_train10 = escalar10.fit_transform(X_train10)
X_test10 = escalar10.transform(X_test10)

In [1170]:
#define el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo10= LogisticRegression()

In [1172]:
algoritmo10.fit(X_train10, y_train10)
#entrena el modelo

LogisticRegression()

In [1173]:
y_pred10 = algoritmo10.predict(X_test10)
y_pred10

array([0, 0, 0, ..., 0, 0, 0], shape=(1453,))

In [1174]:
from sklearn.metrics import confusion_matrix
matriz10 = confusion_matrix(y_test10, y_pred10)
print('Matriz de confusión: ')
print(matriz10)

Matriz de confusión: 
[[848   0]
 [605   0]]


In [1175]:
#precision del modelo 
from sklearn.metrics import precision_score

precision10 = precision_score(y_test10, y_pred10, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision10)

Precisión del modelo:
0.5836200963523744


In [1176]:
#Calculamos la exactitud del modelo 
from sklearn.metrics import accuracy_score

exactitud10 = accuracy_score(y_test10, y_pred10)
print('Exactitud del modelo:')
print(exactitud10)

Exactitud del modelo:
0.5836200963523744


In [1177]:
#Calculando la sensibilidad del modelo 
from sklearn.metrics import recall_score

sensibilidad10 = recall_score(y_test10, y_pred10, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad10)

Sensibilidad del modelo:
1.0
